# Chapter 3 Exercises

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
import arviz as az

In [2]:
az.style.use('arviz-darkgrid')

## Question 1
***

The priors in this model are

\begin{eqnarray}
\mu \text{~} Normal(0,10) \newline
\sigma \text{~} \left|HalfNormal(25) \right|
\end{eqnarray}


The likelihood in our model is  
$$ Normal(\mu, \sigma)$$

And the posterior is

$$ y_i $$

## Question 2
***

There are two parameters in this model $\mu$ and $\sigma$